# Features to replace crs_arr_time and crs_dep_time

In [1]:
import sys
sys.path.append("../..")  # <-  This should point to the root directory of the project relative to this file

from custom_scripts import preprocessing

import math
import numpy as np
import pandas as pd

In [2]:
raw_test_flights = preprocessing.get_test_flights()
average_delays = pd.read_csv('../../data/preprocessing/averages_by_fl_num.csv')


In [3]:
test_flights = raw_test_flights.copy()
print(test_flights.shape)
test_flights.head(1)

(150623, 9)


,fl_date,op_unique_carrier,op_carrier_fl_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance
0,2020-01-01,WN,5888,ONT,SFO,1810,1945,95,363


In [37]:
### change 2400 to 0000 to avoid nans
test_flights['crs_arr_time'].replace({2400:0000},inplace=True)
test_flights['crs_dep_time'].replace({2400:0000},inplace=True)
### pad values
test_flights['arrival_time'] = test_flights['crs_arr_time'].astype(str).str.zfill(4)
test_flights['departure_time'] = test_flights['crs_dep_time'].astype(str).str.zfill(4)
### convert to datetime
test_flights['arrival_time'] = pd.to_datetime(test_flights['arrival_time'], format = '%H%M', errors='coerce')
test_flights['departure_time'] = pd.to_datetime(test_flights['departure_time'], format = '%H%M', errors='coerce')
### convert to minute of day
test_flights['arrival_time'] = test_flights['arrival_time'].dt.hour * 60 + test_flights['arrival_time'].dt.minute
test_flights['departure_time'] = test_flights['departure_time'].dt.hour * 60 + test_flights['departure_time'].dt.minute
### normalize with 2*PI
test_flights['arrival_time'] = 2 * math.pi * test_flights['arrival_time'] / test_flights['arrival_time'].max()
test_flights['departure_time'] = 2 * math.pi * test_flights['departure_time'] / test_flights['departure_time'].max()

In [38]:
test_flights['arr_time_sin'] = np.sin(test_flights['arrival_time'])
test_flights['arr_time_cos'] = np.cos(test_flights['arrival_time'])

test_flights['dep_time_sin'] = np.sin(test_flights['departure_time'])
test_flights['dep_time_cos'] = np.cos(test_flights['departure_time'])

test_flights.drop(['arrival_time','departure_time'],1,inplace=True)

In [39]:
test_flights[test_flights.isna().any(axis=1)]

,fl_date,op_unique_carrier,op_carrier_fl_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_time_sin,arr_time_cos,dep_time_sin,dep_time_cos


In [20]:
2 * math.pi * 2400 / 2400

6.283185307179586